# Pipeline prata: Câmara dos Deputados

Este notebook extrai os dados da fase bronze e popula uma base de dados DuckDB com estes normalizados e deduplicados (ainda sem regularização de chaves estrangeiras)

# 1. Preparação

## 1.1. Imports

In [206]:
import json
import os
import tempfile

from glob import glob
from pathlib import Path

import duckdb
import pandas as pd
import pyarrow as pa
import pyarrow.dataset as ds
import pyarrow.parquet as pq

from dotenv import load_dotenv

%load_ext sql

load_dotenv()

DB_PATH = Path(os.getenv("SILVER_DUCKDB_PATH", ""))
DB_PATH.parent.mkdir(parents=True, exist_ok=True)

TMP_DIR = "/tmp/duckdb_tmp"  # make sure it exists

conn = duckdb.connect(DB_PATH)

conn.execute(f"SET temp_directory='{TMP_DIR}'")
conn.execute("SET memory_limit='50GB'")
conn.execute("SET threads=1")
conn.execute("SET preserve_insertion_order=false")

%sql conn --alias duck

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


## 1.2 Macros SQL

In [207]:
%%sql

CREATE OR REPLACE MACRO jget1(j, p) AS json_extract_string(j, p);

Running query in 'duck'

Count


# 2. Extração inicial

## 2.1 Modelos dimensionais

### 2.1.1 Blocos

In [208]:
%%sql
CREATE OR REPLACE VIEW bronze_camara_blocos AS
SELECT * FROM parquet_scan("data/bronze/snapshots/bronze-2020-2024-v2/camara/blocos/details/year=*/part-*.parquet")
WHERE source = 'camara' AND entity = 'blocos';

SELECT COUNT(*) AS n FROM bronze_camara_blocos;


Running query in 'duck'

n
10


In [209]:
%%sql
DROP TABLE IF EXISTS blocos_camara;
CREATE TABLE blocos_camara AS
WITH base AS (
    SELECT TRY_CAST(payload_json AS JSON) as j, year
    FROM bronze_camara_blocos
)
SELECT
    CAST(jget1(j, '$.dados.id') AS BIGINT) AS id_bloco,
    jget1(j, '$.dados.nome') AS nome,
    CAST(jget1(j, '$.dados.idLegislatura') AS BIGINT) AS id_legislatura,
    jget1(j, '$.dados.uri') AS uri,
    year AS year_snapshot
FROM base
WHERE jget1(j, '$.dados.id') IS NOT NULL;
DROP VIEW IF EXISTS bronze_camara_blocos;

Running query in 'duck'

Success


### 2.1.2 Deputados

In [210]:
%%sql
CREATE OR REPLACE VIEW bronze_camara_deputados AS
SELECT * FROM parquet_scan("data/bronze/snapshots/bronze-2020-2024-v2/camara/deputados/details/year=*/part-*.parquet")
WHERE source = 'camara' AND entity = 'deputados';

SELECT COUNT(*) AS n FROM bronze_camara_deputados;

Running query in 'duck'

n
5970


In [211]:
%%sql
DROP TABLE IF EXISTS deputados_camara;
CREATE TABLE deputados_camara AS
WITH base AS (
    SELECT TRY_CAST(payload_json AS JSON) as j, year
    FROM bronze_camara_deputados
)
SELECT
    CAST(jget1(j, '$.dados.id') AS BIGINT) AS id_deputado,
    jget1(j, '$.dados.nomeCivil') AS nome_civil,
    jget1(j, '$.dados.uri') AS uri,
    year AS year_snapshot
FROM base
WHERE jget1(j, '$.dados.id') IS NOT NULL;
DROP VIEW IF EXISTS bronze_camara_deputados;

Running query in 'duck'

Success


### 2.1.3 Eventos

In [212]:
%%sql
CREATE OR REPLACE VIEW bronze_camara_eventos AS
SELECT * FROM parquet_scan("data/bronze/snapshots/bronze-2020-2024-v2/camara/eventos/details/year=*/part-*.parquet")
WHERE source = 'camara' AND entity = 'eventos';

SELECT COUNT(*) AS n FROM bronze_camara_eventos;

Running query in 'duck'

n
23518


In [213]:

%%sql
DROP TABLE IF EXISTS eventos_camara;
CREATE TABLE eventos_camara AS
WITH base AS (
    SELECT TRY_CAST(payload_json AS JSON) as j, year
    FROM bronze_camara_eventos
)
SELECT
    CAST(jget1(j, '$.dados.id') AS BIGINT) AS id_evento,
    CAST(jget1(j, '$.dados.dataHoraInicio') AS DATETIME) AS data_hora_inicio,
    CAST(jget1(j, '$.dados.dataHoraFim') AS DATETIME) AS data_hora_fim,
    jget1(j, '$.dados.descricao') AS descricao,
    jget1(j, '$.dados.descricaoTipo') AS descricao_tipo,
    jget1(j, '$.dados.fases') AS fases,  -- é sempre null mas deixei por precaução
    jget1(j, '$.dados.uri') AS uri,
    year AS year_snapshot
FROM base
WHERE jget1(j, '$.dados.id') IS NOT NULL;
DROP VIEW IF EXISTS bronze_camara_eventos;

Running query in 'duck'

Success


### 2.1.4 Frentes

In [214]:
%%sql
CREATE OR REPLACE VIEW bronze_camara_frentes AS
SELECT * FROM parquet_scan("data/bronze/snapshots/bronze-2020-2024-v2/camara/frentes/details/year=*/part-*.parquet")
WHERE source = 'camara' AND entity = 'frentes';

SELECT COUNT(*) AS n FROM bronze_camara_frentes;

Running query in 'duck'

n
2518


In [215]:
%%sql
DROP TABLE IF EXISTS frentes_camara;
CREATE TABLE frentes_camara AS
WITH base AS (
    SELECT TRY_CAST(payload_json AS JSON) as j, year
    FROM bronze_camara_frentes
)
SELECT
    CAST(jget1(j, '$.dados.id') AS BIGINT) AS id_frente,
    CAST(jget1(j, '$.dados.coordenador.id') AS BIGINT) AS id_deputado_coordenador,
    CAST(jget1(j, '$.dados.idLegislatura') AS BIGINT) AS id_legislatura,
    jget1(j, '$.dados.titulo') AS titulo,
    jget1(j, '$.dados.uri') AS uri,
    year AS year_snapshot
FROM base
WHERE jget1(j, '$.dados.id') IS NOT NULL;
DROP VIEW IF EXISTS bronze_camara_frentes;

Running query in 'duck'

Success


### 2.1.5 Legislaturas

In [216]:
%%sql
CREATE OR REPLACE VIEW bronze_camara_legislaturas AS
SELECT * FROM parquet_scan("data/bronze/snapshots/bronze-2020-2024-v2/camara/legislaturas/details/year=*/part-*.parquet")
WHERE source = 'camara' AND entity = 'legislaturas';

SELECT COUNT(*) AS n FROM bronze_camara_legislaturas;

Running query in 'duck'

n
114


In [217]:

%%sql
DROP TABLE IF EXISTS legislaturas_camara;
CREATE TABLE legislaturas_camara AS
WITH base AS (
    SELECT TRY_CAST(payload_json AS JSON) as j, year
    FROM bronze_camara_legislaturas
)
SELECT
    CAST(jget1(j, '$.dados.id') AS BIGINT) AS id_legislatura,
    CAST(jget1(j, '$.dados.dataInicio') AS DATE) AS data_inicio,
    CAST(jget1(j, '$.dados.dataFim') AS DATE) AS data_fim,
    jget1(j, '$.dados.uri') AS uri,
    year AS year_snapshot
FROM base
WHERE jget1(j, '$.dados.id') IS NOT NULL;
DROP VIEW IF EXISTS bronze_camara_legislaturas;


Running query in 'duck'

Success


### 2.1.6 Órgaos

In [218]:
%%sql
CREATE OR REPLACE VIEW bronze_camara_orgaos AS
SELECT * FROM parquet_scan("data/bronze/snapshots/bronze-2020-2024-v2/camara/orgaos/details/year=*/part-*.parquet")
WHERE source = 'camara' AND entity = 'orgaos';

SELECT COUNT(*) AS n FROM bronze_camara_orgaos;


Running query in 'duck'

n
2087


In [219]:
%%sql
DROP TABLE IF EXISTS orgaos_camara;
CREATE TABLE orgaos_camara AS
WITH base AS (
    SELECT TRY_CAST(payload_json AS JSON) as j, year
    FROM bronze_camara_orgaos
)
SELECT
    CAST(jget1(j, '$.dados.id') AS BIGINT) AS id_orgao,
    jget1(j, '$.dados.nome') AS nome,
    CAST(jget1(j, '$.dados.codTipoOrgao') AS BIGINT) AS cod_tipo_orgao,
    jget1(j, '$.dados.uri') AS uri,
    year AS year_snapshot
FROM base
WHERE jget1(j, '$.dados.id') IS NOT NULL;
DROP VIEW IF EXISTS bronze_camara_orgaos;


Running query in 'duck'

Success


### 2.1.7 Partidos

In [220]:
%%sql
CREATE OR REPLACE VIEW bronze_camara_partidos AS
SELECT * FROM parquet_scan("data/bronze/snapshots/bronze-2020-2024-v2/camara/partidos/details/year=*/part-*.parquet")
WHERE source = 'camara' AND entity = 'partidos';

SELECT COUNT(*) AS n FROM bronze_camara_partidos;

Running query in 'duck'

n
40


In [221]:
%%sql
DROP TABLE IF EXISTS partidos_camara;
CREATE TABLE partidos_camara AS
WITH base AS (
    SELECT TRY_CAST(payload_json AS JSON) as j, year
    FROM bronze_camara_partidos
)
SELECT
    CAST(jget1(j, '$.dados.id') AS BIGINT) AS id_partido,
    jget1(j, '$.dados.nome') AS nome,
    jget1(j, '$.dados.sigla') AS sigla,
    jget1(j, '$.dados.uri') AS uri,
    year AS year_snapshot
FROM base
WHERE jget1(j, '$.dados.id') IS NOT NULL;
DROP VIEW IF EXISTS bronze_camara_partidos;


Running query in 'duck'

Success


### 2.1.8 Proposições

In [222]:
%%sql
CREATE OR REPLACE VIEW bronze_camara_proposicoes AS
SELECT * FROM parquet_scan("data/bronze/snapshots/bronze-2020-2024-v2/camara/proposicoes/details/year=*/part-*.parquet")
WHERE source = 'camara' AND entity = 'proposicoes';

SELECT COUNT(*) AS n FROM bronze_camara_proposicoes;


Running query in 'duck'

n
81140


In [223]:
%%sql
DROP TABLE IF EXISTS proposicoes_camara;
CREATE TABLE proposicoes_camara AS
WITH base AS (
    SELECT TRY_CAST(payload_json AS JSON) as j, year
    FROM bronze_camara_proposicoes
)
SELECT
    CAST(jget1(j, '$.dados.id') AS BIGINT) AS id_proposicao,
    jget1(j, '$.dados.siglaTipo') as sigla_tipo,
    CAST(jget1(j, '$.dados.numero') AS INTEGER) AS numero,
    CAST(jget1(j, '$.dados.ano') AS INTEGER) AS ano,
    jget1(j, '$.dados.ementa') AS ementa,
    jget1(j, '$.dados.uri') AS uri,
    year AS year_snapshot
FROM base
WHERE jget1(j, '$.dados.id') IS NOT NULL;
DROP VIEW IF EXISTS bronze_camara_proposicoes;

Running query in 'duck'

Success


### 2.1.9 Temas

In [224]:
%%sql
CREATE OR REPLACE VIEW bronze_camara_temas AS
SELECT * FROM parquet_scan("data/bronze/snapshots/bronze-2020-2024-v2/camara/temas/year=*/part-*.parquet")
WHERE source = 'camara' AND entity = 'temas';

SELECT COUNT(*) AS n FROM bronze_camara_temas;

Running query in 'duck'

n
81140


In [225]:
%%sql
DROP TABLE IF EXISTS temas_camara;
CREATE TABLE temas_camara AS
WITH base AS (
    SELECT TRY_CAST(payload_json AS JSON) as j, year
    FROM bronze_camara_temas
),
exploded as (
    SELECT
        CAST(jget1(elem.value, '$.codTema') AS BIGINT) AS id_tema,
        jget1(elem.value, '$.tema') AS descricao,
    FROM base
    CROSS JOIN json_each(json_extract(j, '$.dados')) AS elem
)
SELECT
    id_tema,
    max(descricao) as descricao,
FROM exploded
WHERE id_tema IS NOT NULL
GROUP BY id_tema
ORDER BY id_tema;
DROP VIEW IF EXISTS bronze_camara_temas;

Running query in 'duck'

Success


## 2.2 Modelos factuais

### 2.2.1 Autores

In [226]:
%%sql
CREATE OR REPLACE VIEW bronze_camara_autores AS
SELECT * FROM parquet_scan("data/bronze/snapshots/bronze-2020-2024-v2/camara/autores/year=*/part-*.parquet")
WHERE source = 'camara' AND entity = 'autores';

SELECT COUNT(*) AS n FROM bronze_camara_autores;

Running query in 'duck'

n
81140


In [227]:
%%sql
DROP TABLE IF EXISTS autores_camara;
CREATE TABLE autores_camara AS
WITH base AS (
    SELECT TRY_CAST(payload_json AS JSON) AS j,
    year,
    id AS id_proposicao,
    FROM bronze_camara_autores
),
exploded AS (
    SELECT
        CAST(jget1(elem.value, '$.codTipo') AS BIGINT)        AS cod_tipo,
        jget1(elem.value, '$.uri')                           AS uri,
        CAST(jget1(elem.value, '$.ordemAssinatura') AS INT)  AS ordem_assinatura,
        CAST(jget1(elem.value, '$.proponente') AS BOOLEAN)   AS proponente,
        CAST(b.id_proposicao AS BIGINT)                      AS id_proposicao,
        b.year
    FROM base b
    CROSS JOIN json_each(b.j, '$.dados') AS elem
),
numbered AS (
    SELECT
        ROW_NUMBER() OVER () AS id_autor,   -- surrogate key
        cod_tipo,
        uri,
        ordem_assinatura,
        proponente,
        id_proposicao,
        year
    FROM exploded
)
SELECT *
FROM numbered;
DROP VIEW IF EXISTS bronze_camara_autores;

Running query in 'duck'

Success


### 2.2.2 Orientações

In [228]:
%%sql
CREATE OR REPLACE VIEW bronze_camara_orientacoes AS
SELECT * FROM parquet_scan("data/bronze/snapshots/bronze-2020-2024-v2/camara/orientacoes/year=*/part-*.parquet")
WHERE source = 'camara' AND entity = 'orientacoes';

SELECT COUNT(*) AS n FROM bronze_camara_orientacoes;

Running query in 'duck'

n
38525


In [229]:
%%sql
DROP TABLE IF EXISTS orientacoes_camara;

CREATE TABLE orientacoes_camara AS
WITH base AS (
  SELECT
    TRY_CAST(payload_json AS JSON) AS j,
    year AS year_snapshot,
    id AS id_votacao
  FROM bronze_camara_orientacoes
),
exploded AS (
  SELECT
    CAST(json_extract_string(e.value, '$.codPartidoBloco') AS BIGINT) AS cod_partido_bloco,
    json_extract_string(e.value, '$.codTipoLideranca') AS cod_tipo_lideranca,
    json_extract_string(e.value, '$.orientacaoVoto') AS orientacao_voto,
    json_extract_string(e.value, '$.siglaPartidoBloco') AS sigla_partido_bloco,
    json_extract_string(e.value, '$.uriPartidoBloco') AS uri_partido_bloco,
    b.id_votacao,
    b.year_snapshot
  FROM base b
  CROSS JOIN json_each(b.j, '$.dados') AS e
),
numbered AS (
    SELECT
        ROW_NUMBER() OVER () AS id_orientacao,
        id_votacao,
        sigla_partido_bloco,
        orientacao_voto,
        cod_partido_bloco,
        cod_tipo_lideranca,
        uri_partido_bloco,
        year_snapshot
    FROM exploded
)
SELECT *
FROM numbered;
DROP VIEW IF EXISTS bronze_camara_orientacoes;


Running query in 'duck'

Success


### 2.2.3 Tramitações

In [230]:
%%sql
-- View over the Parquet snapshot
CREATE OR REPLACE VIEW bronze_camara_tramitacoes AS
SELECT *
FROM parquet_scan('data/bronze/snapshots/bronze-2020-2024-v2/camara/tramitacoes/year=*/part-*.parquet')
WHERE source = 'camara' AND entity = 'tramitacoes';

-- Materialize exploded tramitacoes
DROP TABLE IF EXISTS tramitacoes_camara;

CREATE TABLE tramitacoes_camara AS
WITH base AS (
  SELECT
    TRY_CAST(payload_json AS JSON) AS j,
    year AS year_snapshot,
    id   AS id_proposicao
  FROM bronze_camara_tramitacoes
),
exploded AS (
  SELECT
    CAST(b.id_proposicao AS BIGINT)                                        AS id_proposicao,
    json_extract_string(e.value, '$.ambito')                               AS ambito,
    json_extract_string(e.value, '$.apreciacao')                           AS apreciacao,
    json_extract_string(e.value, '$.codSituacao')                          AS cod_situacao,
    json_extract_string(e.value, '$.codTipoTramitacao')                    AS cod_tipo_tramitacao,
    CAST(json_extract_string(e.value, '$.dataHora') AS TIMESTAMP)          AS data_hora,
    json_extract_string(e.value, '$.descricaoSituacao')                    AS descricao_situacao,
    json_extract_string(e.value, '$.descricaoTramitacao')                  AS descricao_tramitacao,
    json_extract_string(e.value, '$.despacho')                             AS despacho,
    json_extract_string(e.value, '$.regime')                               AS regime,
    CAST(json_extract_string(e.value, '$.sequencia') AS INTEGER)           AS sequencia,
    json_extract_string(e.value, '$.siglaOrgao')                           AS sigla_orgao,
    json_extract_string(e.value, '$.uriOrgao')                             AS uri_orgao,
    json_extract_string(e.value, '$.uriUltimoRelator')                     AS uri_ultimo_relator,
    b.year_snapshot                                                         AS year_snapshot
  FROM base b
  CROSS JOIN json_each(b.j, '$.dados') AS e
),
numbered AS (
    SELECT
        ROW_NUMBER() OVER () AS id_tramitacao,
        id_proposicao,
        ambito,
        apreciacao,
        cod_situacao,
        cod_tipo_tramitacao,
        data_hora,
        descricao_situacao,
        descricao_tramitacao,
        despacho,
        regime,
        sequencia,
        sigla_orgao,
        uri_orgao,
        uri_ultimo_relator,
        year_snapshot
    FROM exploded
)
SELECT *
FROM numbered;


DROP VIEW IF EXISTS bronze_camara_tramitacoes;


Running query in 'duck'

Success


### 2.2.4 Votações

In [231]:
%%sql
CREATE OR REPLACE VIEW bronze_camara_votacoes AS
SELECT * FROM parquet_scan("data/bronze/snapshots/bronze-2020-2024-v2/camara/votacoes/year=*/part-*.parquet")
WHERE source = 'camara' AND entity = 'votacoes';

SELECT COUNT(*) AS n FROM bronze_camara_votacoes;

Running query in 'duck'

n
81140


In [232]:
%%sql
DROP TABLE IF EXISTS votacoes_camara;

CREATE TABLE votacoes_camara AS
WITH base AS (
  SELECT
    TRY_CAST(payload_json AS JSON) AS j,
    year as year_snapshot,
    id AS id_proposicao
  FROM bronze_camara_votacoes
),
exploded AS (
  SELECT
    CAST(json_extract_string(e.value, '$.aprovacao') AS BOOLEAN) AS aprovacao,
    CAST(json_extract_string(e.value, '$.data') AS DATE) AS data,
    json_extract_string(e.value, '$.descricao') AS descricao,
    json_extract_string(e.value, '$.id') AS id_votacao, -- como text mesmo, por conta do hífen
    json_extract_string(e.value, '$.uri') AS uri,
    json_extract_string(e.value, '$.uriEvento') AS uri_evento,
    json_extract_string(e.value, '$.uriOrgao') AS uri_orgao,
    CAST(b.id_proposicao AS BIGINT) AS id_proposicao,
    b.year_snapshot
  FROM base b
  CROSS JOIN json_each(b.j, '$.dados') AS e
)
SELECT
    id_votacao,
    id_proposicao,
    data,
    descricao,
    aprovacao,
    uri_evento,
    uri_orgao,
    uri,
    year_snapshot,
FROM exploded;


Running query in 'duck'

Count
38712


### 2.2.5 Votos

In [233]:
with tempfile.NamedTemporaryFile() as tf:
    paths = glob("data/bronze/snapshots/bronze-2020-2024-v2/camara/votos/year=*/part-*.parquet")
    table = pq.read_table(paths)
    df = table.to_pandas()

    exploded_rows = []
    for _, row in df.iterrows():
        payload = json.loads(row["payload_json"])
        for item in payload["dados"]:
            exploded_rows.append({
                "id_votacao": row["id"],
                "id_deputado": item["deputado_"]["id"],
                "tipo_voto": item["tipoVoto"],
                "data_hora": item["dataRegistroVoto"],
                "year_snapshot": row["year"],
            })

    exploded_df = pd.DataFrame(exploded_rows)
    exploded_df.to_csv(tf.name, index=False)

    conn.execute("""
        DROP TABLE IF EXISTS votos_camara;

        -- Create table with PK declared up-front
        CREATE TABLE votos_camara (
            id_voto BIGINT PRIMARY KEY,
            id_votacao TEXT,
            id_deputado BIGINT,
            tipo_voto TEXT,
            data_hora TIMESTAMP,
            year_snapshot INTEGER
        );

        -- Deterministic incremental ID via ROW_NUMBER with ORDER BY
        INSERT INTO votos_camara
        SELECT
            ROW_NUMBER() OVER (ORDER BY id_votacao, id_deputado, data_hora, year_snapshot) AS id_voto,
            CAST(id_votacao AS TEXT)                           AS id_votacao,
            CAST(id_deputado AS BIGINT)                        AS id_deputado,
            CAST(tipo_voto AS TEXT)                            AS tipo_voto,
            CAST(data_hora AS TIMESTAMP)                       AS data_hora,
            CAST(year_snapshot AS INTEGER)                     AS year_snapshot
        FROM read_csv_auto($path, HEADER=TRUE);
    """, {"path": tf.name})


## 2.3 Outras relações entre tabelas

### 2.3.1 Blocos e partidos

In [234]:
%%sql
CREATE OR REPLACE VIEW bronze_camara_blocos_partidos AS
SELECT * FROM parquet_scan("data/bronze/snapshots/bronze-2020-2024-v2/camara/blocos/partidos/year=*/part-*.parquet")
WHERE source = 'camara' AND entity = 'blocos/partidos';

SELECT COUNT(*) AS n FROM bronze_camara_blocos_partidos;

Running query in 'duck'

n
5


In [235]:
%%sql
DROP TABLE IF EXISTS blocos_partidos_camara;

CREATE TABLE blocos_partidos_camara AS
WITH base AS (
  SELECT
    TRY_CAST(payload_json AS JSON) AS j,
    year AS year_snapshot,
    id AS id_bloco
  FROM bronze_camara_blocos_partidos
),
exploded AS (
  SELECT
    CAST(json_extract_string(e.value, '$.id') AS BIGINT) AS id_partido,
    CAST(b.id_bloco AS BIGINT) AS id_bloco,
    b.year_snapshot
  FROM base b
  CROSS JOIN json_each(b.j, '$.dados') AS e
),
numbered AS (
    SELECT
        ROW_NUMBER() OVER () AS id_bloco_partido,
        id_bloco,
        id_partido,
        year_snapshot
    FROM exploded
)
SELECT *
FROM numbered;


Running query in 'duck'

Count
22


### 2.3.2 Deputados e frentes

In [236]:
%%sql
CREATE OR REPLACE VIEW bronze_camara_deputados_frentes AS
SELECT * FROM parquet_scan("data/bronze/snapshots/bronze-2020-2024-v2/camara/deputados/frentes/year=*/part-*.parquet")
WHERE source = 'camara' AND entity = 'deputados/frentes';

SELECT COUNT(*) AS n FROM bronze_camara_deputados_frentes;

Running query in 'duck'

n
1435


In [237]:
%%sql
DROP TABLE IF EXISTS deputados_frentes_camara;

CREATE TABLE deputados_frentes_camara AS
WITH base AS (
  SELECT
    TRY_CAST(payload_json AS JSON) AS j,
    year AS year_snapshot,
    id AS id_deputado
  FROM bronze_camara_deputados_frentes
),
exploded AS (
  SELECT
    CAST(json_extract_string(e.value, '$.id') AS BIGINT) AS id_frente,
    CAST(b.id_deputado AS BIGINT) AS id_deputado,
    b.year_snapshot
  FROM base b
  CROSS JOIN json_each(b.j, '$.dados') AS e
),
numbered AS (
    SELECT
        ROW_NUMBER() OVER () AS id_deputado_frente,
        id_deputado,
        id_frente,
        year_snapshot
    FROM exploded
)
SELECT *
FROM numbered;

Running query in 'duck'

Count
146271


### 2.3.3 Deputados e histórico

In [238]:
%%sql
CREATE OR REPLACE VIEW bronze_camara_deputados_historico AS
SELECT * FROM parquet_scan("data/bronze/snapshots/bronze-2020-2024-v2/camara/deputados/historico/year=*/part-*.parquet")
WHERE source = 'camara' AND entity = 'deputados/historico';

SELECT COUNT(*) AS n FROM bronze_camara_deputados_historico;

Running query in 'duck'

n
512


In [239]:
%%sql
DROP TABLE IF EXISTS deputados_historico_camara;

CREATE TABLE deputados_historico_camara AS
WITH base AS (
  SELECT
    TRY_CAST(payload_json AS JSON) AS j,
    year AS year_snapshot,
    id AS id_deputado
  FROM bronze_camara_deputados_historico
),
exploded AS (
  SELECT
    json_extract_string(e.value, '$.condicaoEleitoral') AS condicao_eleitoral,
    CAST(json_extract_string(e.value, '$.dataHora') AS DATETIME) AS data_hora,
    json_extract_string(e.value, '$.descricaoStatus') AS descricao_status,
    CAST(json_extract_string(e.value, '$.idLegislatura') AS BIGINT) AS id_legislatura,
    json_extract_string(e.value, '$.situacao') AS situacao,
    CAST(b.id_deputado AS BIGINT) AS id_deputado,
    b.year_snapshot
  FROM base b
  CROSS JOIN json_each(b.j, '$.dados') AS e
),
numbered AS (
    SELECT
        ROW_NUMBER() OVER () AS id_deputado_historico,
        id_deputado,
        id_legislatura,
        data_hora,
        condicao_eleitoral,
        descricao_status,
        year_snapshot
    FROM exploded
)
SELECT *
FROM numbered;


Running query in 'duck'

Count
4507


### 2.3.4 Deputados e órgãos

In [240]:
%%sql
CREATE OR REPLACE VIEW bronze_camara_deputados_orgaos AS
SELECT * FROM parquet_scan("data/bronze/snapshots/bronze-2020-2024-v2/camara/deputados/orgaos/year=*/part-*.parquet")
WHERE source = 'camara' AND entity = 'deputados/orgaos';

SELECT COUNT(*) AS n FROM bronze_camara_deputados_orgaos;

Running query in 'duck'

n
512


In [241]:
%%sql
DROP TABLE IF EXISTS deputados_orgaos_camara;

CREATE TABLE deputados_orgaos_camara AS
WITH base AS (
  SELECT
    TRY_CAST(payload_json AS JSON) AS j,
    year AS year_snapshot,
    id AS id_deputado
  FROM bronze_camara_deputados_orgaos
),
exploded AS (
  SELECT
    CAST(json_extract_string(e.value, '$.idOrgao') AS INTEGER) AS id_orgao,
    CAST(json_extract_string(e.value, '$.codTitulo') AS INTEGER) AS cod_titulo,
    CAST(json_extract_string(e.value, '$.dataInicio') AS DATETIME) AS data_inicio,
    CAST(json_extract_string(e.value, '$.dataFim') AS DATETIME) AS data_fim,
    CAST(b.id_deputado AS BIGINT) AS id_deputado,
    b.year_snapshot
  FROM base b
  CROSS JOIN json_each(b.j, '$.dados') AS e
),
numbered AS (
    SELECT
        ROW_NUMBER() OVER () AS id_deputado_orgao,
        id_deputado,
        id_orgao,
        cod_titulo,
        data_inicio,
        data_fim,
        year_snapshot
    FROM exploded
)
SELECT *
FROM numbered;

Running query in 'duck'

Count
3462


### 2.3.5 Eventos e órgaos

In [242]:
%%sql
CREATE OR REPLACE VIEW bronze_camara_eventos_orgaos AS
SELECT * FROM parquet_scan("data/bronze/snapshots/bronze-2020-2024-v2/camara/eventos/orgaos/year=*/part-*.parquet")
WHERE source = 'camara' AND entity = 'eventos/orgaos';

SELECT COUNT(*) AS n FROM bronze_camara_eventos_orgaos;

Running query in 'duck'

n
11759


In [243]:
%%sql
DROP TABLE IF EXISTS eventos_orgaos_camara;

CREATE TABLE eventos_orgaos_camara AS
WITH base AS (
  SELECT
    TRY_CAST(payload_json AS JSON) AS j,
    year AS year_snapshot,
    id AS id_evento
  FROM bronze_camara_eventos_orgaos
),
exploded AS (
  SELECT
    CAST(b.id_evento AS BIGINT) AS id_evento,
    CAST(json_extract_string(e.value, '$.id') AS INTEGER) AS id_orgao,
    b.year_snapshot
  FROM base b
  CROSS JOIN json_each(b.j, '$.dados') AS e
),
numbered AS (
    SELECT 
        ROW_NUMBER() OVER () AS id_evento_orgao,
        id_evento, id_orgao, year_snapshot
    FROM exploded
)
SELECT * FROM numbered;

Running query in 'duck'

Count
12118


### 2.3.6 Eventos e pautas

In [244]:
%%sql
CREATE OR REPLACE VIEW bronze_camara_eventos_pauta AS
SELECT * FROM parquet_scan("data/bronze/snapshots/bronze-2020-2024-v2/camara/eventos/pauta/year=*/part-*.parquet")
WHERE source = 'camara' AND entity = 'eventos/pauta';

SELECT COUNT(*) AS n FROM bronze_camara_eventos_pauta;

Running query in 'duck'

n
11759


In [245]:
with tempfile.NamedTemporaryFile() as tf:
    paths = glob("data/bronze/snapshots/bronze-2020-2024-v2/camara/eventos/pauta/year=*/part-*.parquet")
    table = pq.read_table(paths)
    df = table.to_pandas()

    exploded_rows = []
    for _, row in df.iterrows():
        payload = json.loads(row["payload_json"])
        for item in payload["dados"]:
            exploded_rows.append({
                "id_evento": row["id"],
                "cod_regime": item.get("codRegime"),
                "ordem": item.get("ordem"),
                "id_proposicao": (item.get("proposicao_", {}) or {}).get("id"),
                "id_relator": (item.get("relator") or {}).get("id") if item.get("relator") else None,
                "year_snapshot": row["year"],
            })

    exploded_df = pd.DataFrame(exploded_rows)
    exploded_df.to_csv(tf.name, index=False)

    conn.execute("""
        DROP TABLE IF EXISTS eventos_pauta_camara;

        -- Create table with surrogate PK
        CREATE TABLE eventos_pauta_camara (
            id_pauta BIGINT PRIMARY KEY,
            id_evento BIGINT,
            cod_regime TEXT,
            ordem INTEGER,
            id_proposicao BIGINT,
            id_relator BIGINT,
            year_snapshot INTEGER
        );

        -- Deterministic incremental ID via ROW_NUMBER
        INSERT INTO eventos_pauta_camara
        SELECT
            ROW_NUMBER() OVER (
                ORDER BY
                    CAST(id_evento AS BIGINT),
                    CAST(ordem AS INTEGER),
                    CAST(id_proposicao AS BIGINT),
                    COALESCE(TRY_CAST(id_relator AS BIGINT), -1),
                    CAST(year_snapshot AS INTEGER)
            ) AS id_pauta,
            CAST(id_evento AS BIGINT)                     AS id_evento,
            CAST(cod_regime AS TEXT)                      AS cod_regime,
            CAST(ordem AS INTEGER)                        AS ordem,
            CAST(id_proposicao AS BIGINT)                 AS id_proposicao,
            TRY_CAST(id_relator AS BIGINT)                AS id_relator,   -- stays NULL if absent
            CAST(year_snapshot AS INTEGER)                AS year_snapshot
        FROM read_csv_auto($path, HEADER=TRUE);
    """, {"path": tf.name})


### 2.3.7 Legislaturas e líderes

In [246]:
with tempfile.NamedTemporaryFile() as tf:
    paths = glob("data/bronze/snapshots/bronze-2020-2024-v2/camara/legislaturas/lideres/year=*/part-*.parquet")
    table = pq.read_table(paths)
    df = table.to_pandas()

    exploded_rows = []
    for _, row in df.iterrows():
        payload = json.loads(row["payload_json"])
        for item in payload["dados"]:
            exploded_rows.append({
                "id_legislatura": row["id"],
                "nome_bancada": item["bancada"]["nome"],
                "tipo_bancada": item["bancada"]["tipo"],
                "uri_bancada": item["bancada"]["uri"],
                "data_inicio": item["dataInicio"],
                "data_fim": item["dataFim"],
                "id_deputado": item["parlamentar"]["id"],
                "titulo": item["titulo"],
                "year_snapshot": row["year"],
            })

    exploded_df = pd.DataFrame(exploded_rows)
    exploded_df.to_csv(tf.name, index=False)

    conn.execute("""
        DROP TABLE IF EXISTS legislaturas_lideres_camara;

        -- Create with surrogate PK
        CREATE TABLE legislaturas_lideres_camara (
            id_lider BIGINT PRIMARY KEY,
            id_legislatura BIGINT,
            nome_bancada TEXT,
            tipo_bancada TEXT,
            uri_bancada TEXT,
            data_inicio TIMESTAMP,
            data_fim TIMESTAMP,
            id_deputado BIGINT,
            titulo TEXT,
            year_snapshot INTEGER
        );

        -- Insert with deterministic incremental ID
        INSERT INTO legislaturas_lideres_camara
        SELECT
            ROW_NUMBER() OVER (
                ORDER BY
                    CAST(id_legislatura AS BIGINT),
                    COALESCE(nome_bancada, ''),
                    COALESCE(tipo_bancada, ''),
                    COALESCE(id_deputado, -1),
                    CAST(year_snapshot AS INTEGER)
            ) AS id_lider,
            CAST(id_legislatura AS BIGINT)         AS id_legislatura,
            CAST(nome_bancada AS TEXT)             AS nome_bancada,
            CAST(tipo_bancada AS TEXT)             AS tipo_bancada,
            CAST(uri_bancada AS TEXT)              AS uri_bancada,
            CAST(data_inicio AS TIMESTAMP)         AS data_inicio,
            CAST(data_fim AS TIMESTAMP)            AS data_fim,
            CAST(id_deputado AS BIGINT)            AS id_deputado,
            CAST(titulo AS TEXT)                   AS titulo,
            CAST(year_snapshot AS INTEGER)         AS year_snapshot
        FROM read_csv_auto($path, HEADER=TRUE);
    """, {"path": tf.name})


### 2.3.8 Legislaturas e mesa

In [247]:

%%sql
CREATE OR REPLACE VIEW bronze_camara_legislaturas_mesa AS
SELECT * FROM parquet_scan("data/bronze/snapshots/bronze-2020-2024-v2/camara/legislaturas/mesa/year=*/part-*.parquet")
WHERE source = 'camara' AND entity = 'legislaturas/mesa';

SELECT COUNT(*) AS n FROM bronze_camara_legislaturas_mesa;

Running query in 'duck'

n
57


In [248]:

%%sql
DROP TABLE IF EXISTS legislaturas_mesa_camara;
CREATE TABLE legislaturas_mesa_camara AS
WITH base AS (
    SELECT TRY_CAST(payload_json AS JSON) as j, 
    year AS year_snapshot,
    id AS id_legislatura,
    FROM bronze_camara_legislaturas_mesa
),
exploded as (
    SELECT
        CAST(base.id_legislatura AS BIGINT) AS id_legislatura,
        -- id
        CAST(jget1(elem.value, '$.id') AS BIGINT) AS id_deputado,
        -- codTitulo
        jget1(elem.value, '$.codTitulo') AS cod_titulo,
        -- dataInicio
        CAST(jget1(elem.value, '$.dataInicio') AS DATE) AS data_inicio,
        -- dataFim
        CAST(jget1(elem.value, '$.dataFim') AS DATE) AS data_fim,
        year_snapshot,
    FROM base
    CROSS JOIN json_each(json_extract(j, '$.dados')) AS elem
),
numbered AS (
    SELECT
        ROW_NUMBER() OVER () AS id_legislatura_mesa,
        id_legislatura,
        id_deputado,
        cod_titulo,
        data_inicio,
        data_fim,
        year_snapshot
    FROM exploded
)
SELECT *
FROM numbered
WHERE id_legislatura IS NOT NULL AND id_deputado IS NOT NULL
ORDER BY id_legislatura, cod_titulo;




Running query in 'duck'

Count
144


### 2.3.9 Partidos e líderes

In [249]:
%%sql
CREATE OR REPLACE VIEW bronze_partidos_lideres AS
SELECT * FROM parquet_scan("data/bronze/snapshots/bronze-2020-2024-v2/camara/partidos/lideres/year=*/part-*.parquet")
WHERE source = 'camara' AND entity = 'partidos/lideres';

SELECT COUNT(*) AS n FROM bronze_partidos_lideres;


Running query in 'duck'

n
20


In [250]:
%%sql
DROP TABLE IF EXISTS partidos_lideres_camara;

CREATE TABLE partidos_lideres_camara AS
WITH base AS (
  SELECT
    TRY_CAST(payload_json AS JSON) AS j,
    year AS year_snapshot,
    id AS id_partido,
  FROM bronze_partidos_lideres
),
exploded AS (
  SELECT
    CAST(b.id_partido AS BIGINT) AS id_partido,
      -- codTitulo
    json_extract_string(e.value, '$.codTitulo') AS cod_titulo,
    -- dataInicio
    CAST(json_extract_string(e.value, '$.dataInicio') AS DATE) AS data_inicio,
    -- dataFim
    CAST(json_extract_string(e.value, '$.dataFim') AS DATE) AS data_fim,
    -- id (deputado)
    CAST(json_extract_string(e.value, '$.id') AS BIGINT) AS id_deputado,
    -- year
    b.year_snapshot,
  FROM base b
  CROSS JOIN json_each(b.j, '$.dados') AS e
),
numbered AS (
    SELECT
        ROW_NUMBER() OVER () AS id_partido_lider,
        id_partido,
        cod_titulo,
        data_inicio,
        data_fim,
        id_deputado,
        year_snapshot
    FROM exploded
)
SELECT *
FROM numbered;
DROP VIEW IF EXISTS bronze_partidos_lideres;

Running query in 'duck'

Success


### 2.3.10 Partidos e membros

In [251]:
%%sql
CREATE OR REPLACE VIEW bronze_partidos_membros AS
SELECT * FROM parquet_scan("data/bronze/snapshots/bronze-2020-2024-v2/camara/partidos/membros/year=*/part-*.parquet")
WHERE source = 'camara' AND entity = 'partidos/membros';

SELECT COUNT(*) AS n FROM bronze_partidos_membros;

Running query in 'duck'

n
20


In [252]:
%%sql
DROP TABLE IF EXISTS partidos_membros_camara;

CREATE TABLE partidos_membros_camara AS
WITH base AS (
  SELECT
    TRY_CAST(payload_json AS JSON) AS j,
    year AS year_snapshot,
    id AS id_partido,
  FROM bronze_partidos_membros
),
exploded AS (
  SELECT
    CAST(b.id_partido AS BIGINT) AS id_partido,
    CAST(json_extract_string(e.value, '$.id') AS BIGINT) AS id_deputado,
    CAST(json_extract_string(e.value, '$.id_legislatura') AS INTEGER) AS id_legislatura,
    b.year_snapshot,
  FROM base b
  CROSS JOIN json_each(b.j, '$.dados') AS e
),
numbered AS (
    SELECT
        ROW_NUMBER() OVER () AS id_partido_membro,
        id_partido,
        id_deputado,
        id_legislatura,
        year_snapshot
    FROM exploded
)
SELECT *
FROM numbered;
DROP VIEW IF EXISTS bronze_partidos_membros;

Running query in 'duck'

Success


# 3. Regularização

## 3.1. Normalização de PKs

Aqui vamos determinar as chaves primárias (PK) de cada tabela, deduplicando onde necessário.

### 3.1.2 Blocos

In [253]:
%%sql
CREATE TABLE blocos_camara_dedup AS
SELECT *
FROM (
    SELECT *,
           ROW_NUMBER() OVER (PARTITION BY id_bloco ORDER BY year_snapshot DESC) AS rn
    FROM blocos_camara
)
WHERE rn = 1;

DROP TABLE blocos_camara;
ALTER TABLE blocos_camara_dedup RENAME TO blocos_camara;

Running query in 'duck'

Success


In [254]:
%%sql
ALTER TABLE blocos_camara
    ALTER COLUMN id_bloco TYPE BIGINT;

ALTER TABLE blocos_camara
    ADD CONSTRAINT pk_blocos PRIMARY KEY (id_bloco);

Running query in 'duck'

Success


### 3.1.3 Deputados

In [255]:
%%sql
CREATE TABLE deputados_camara_dedup AS
SELECT *
FROM (
    SELECT *,
           ROW_NUMBER() OVER (PARTITION BY id_deputado ORDER BY year_snapshot DESC) AS rn
    FROM deputados_camara
)
WHERE rn = 1;

DROP TABLE deputados_camara;
ALTER TABLE deputados_camara_dedup RENAME TO deputados_camara;


Running query in 'duck'

Success


In [256]:
%%sql
ALTER TABLE deputados_camara
    ALTER COLUMN id_deputado TYPE BIGINT;

ALTER TABLE deputados_camara
    ADD CONSTRAINT pk_deputados PRIMARY KEY (id_deputado);

Running query in 'duck'

Success


### 3.1.4 Eventos

In [257]:
%%sql
CREATE TABLE eventos_camara_dedup AS
SELECT *
FROM (
    SELECT *,
           ROW_NUMBER() OVER (PARTITION BY id_evento ORDER BY year_snapshot DESC) AS rn
    FROM eventos_camara
)
WHERE rn = 1;

DROP TABLE eventos_camara;
ALTER TABLE eventos_camara_dedup RENAME TO eventos_camara;

Running query in 'duck'

Success


In [258]:
%%sql
ALTER TABLE eventos_camara
    ALTER COLUMN id_evento TYPE BIGINT;

ALTER TABLE eventos_camara
    ADD CONSTRAINT pk_eventos PRIMARY KEY (id_evento);

Running query in 'duck'

Success


### 3.1.5 Frentes

In [259]:
%%sql
CREATE TABLE frentes_camara_dedup AS
SELECT *
FROM (
    SELECT *,
           ROW_NUMBER() OVER (PARTITION BY id_frente ORDER BY year_snapshot DESC) AS rn
    FROM frentes_camara
)
WHERE rn = 1;

DROP TABLE frentes_camara;
ALTER TABLE frentes_camara_dedup RENAME TO frentes_camara;

Running query in 'duck'

Success


In [260]:
%%sql
ALTER TABLE frentes_camara
    ALTER COLUMN id_frente TYPE BIGINT;

ALTER TABLE frentes_camara
    ADD CONSTRAINT pk_frentes PRIMARY KEY (id_frente);

Running query in 'duck'

Success


### 3.1.6 Legislaturas

In [261]:
%%sql
CREATE TABLE legislaturas_camara_dedup AS
SELECT *
FROM (
    SELECT *,
           ROW_NUMBER() OVER (PARTITION BY id_legislatura ORDER BY year_snapshot DESC) AS rn
    FROM legislaturas_camara
)
WHERE rn = 1;

DROP TABLE legislaturas_camara;
ALTER TABLE legislaturas_camara_dedup RENAME TO legislaturas_camara;

Running query in 'duck'

Success


In [262]:
%%sql
ALTER TABLE legislaturas_camara
    ALTER COLUMN id_legislatura TYPE BIGINT;

ALTER TABLE legislaturas_camara
    ADD CONSTRAINT pk_legislaturas PRIMARY KEY (id_legislatura);

Running query in 'duck'

Success


### 3.1.7 Órgãos

In [263]:
%%sql
CREATE TABLE orgaos_camara_dedup AS
SELECT *
FROM (
    SELECT *,
           ROW_NUMBER() OVER (PARTITION BY id_orgao ORDER BY year_snapshot DESC) AS rn
    FROM orgaos_camara
)
WHERE rn = 1;

DROP TABLE orgaos_camara;
ALTER TABLE orgaos_camara_dedup RENAME TO orgaos_camara;

Running query in 'duck'

Success


In [264]:
%%sql
ALTER TABLE orgaos_camara
    ALTER COLUMN id_orgao TYPE BIGINT;

ALTER TABLE orgaos_camara
    ADD CONSTRAINT pk_orgaos PRIMARY KEY (id_orgao);

Running query in 'duck'

Success


### 3.1.8 Partidos

In [265]:


%%sql
CREATE TABLE partidos_camara_dedup AS
SELECT *
FROM (
    SELECT *,
           ROW_NUMBER() OVER (PARTITION BY id_partido ORDER BY year_snapshot DESC) AS rn
    FROM partidos_camara
)
WHERE rn = 1;

DROP TABLE partidos_camara;
ALTER TABLE partidos_camara_dedup RENAME TO partidos_camara;

Running query in 'duck'

Success


In [266]:
%%sql
ALTER TABLE partidos_camara
    ALTER COLUMN id_partido TYPE BIGINT;

ALTER TABLE partidos_camara
    ADD CONSTRAINT pk_partidos PRIMARY KEY (id_partido);

Running query in 'duck'

Success


### 3.1.9 Proposições

In [267]:
%%sql
ALTER TABLE proposicoes_camara
    ALTER COLUMN id_proposicao TYPE BIGINT;

ALTER TABLE proposicoes_camara
    ADD CONSTRAINT pk_proposicoes PRIMARY KEY (id_proposicao);

Running query in 'duck'

Success


### 3.1.10 Temas

In [268]:
%%sql
ALTER TABLE temas_camara
    ALTER COLUMN id_tema TYPE BIGINT;

ALTER TABLE temas_camara
    ADD CONSTRAINT pk_temas PRIMARY KEY (id_tema);


Running query in 'duck'

Success


### 3.1.11 Autores

In [269]:
%%sql
ALTER TABLE autores_camara
    ALTER COLUMN id_proposicao TYPE BIGINT;

ALTER TABLE autores_camara
    ALTER COLUMN ordem_assinatura TYPE INTEGER;

Running query in 'duck'

Success


In [270]:
%%sql
ALTER TABLE autores_camara
    ADD CONSTRAINT pk_autores PRIMARY KEY (id_autor);


Running query in 'duck'

Success


### 3.1.12 Orientações

In [271]:
%%sql
ALTER TABLE orientacoes_camara
    ALTER COLUMN id_orientacao TYPE BIGINT;

ALTER TABLE orientacoes_camara
    ADD CONSTRAINT pk_orientacoes
    PRIMARY KEY (id_orientacao);

Running query in 'duck'

Success


### 3.1.13 Tramitações

In [272]:
%%sql
ALTER TABLE tramitacoes_camara
    ALTER COLUMN id_tramitacao TYPE BIGINT;

ALTER TABLE tramitacoes_camara
    ADD CONSTRAINT pk_tramitacoes
    PRIMARY KEY (id_tramitacao);

Running query in 'duck'

Success


### 3.1.14 Votações

In [273]:
%%sql
CREATE TABLE votacoes_camara_dedup AS
SELECT *
FROM (
    SELECT *,
           ROW_NUMBER() OVER (PARTITION BY id_votacao ORDER BY year_snapshot DESC) AS rn
    FROM votacoes_camara
)
WHERE rn = 1;

DROP TABLE votacoes_camara;
ALTER TABLE votacoes_camara_dedup RENAME TO votacoes_camara;

ALTER TABLE votacoes_camara
    ALTER COLUMN id_votacao TYPE TEXT;

ALTER TABLE votacoes_camara
    ADD CONSTRAINT pk_votacoes
    PRIMARY KEY (id_votacao);

Running query in 'duck'

Success


### 3.1.15 Blocos e partidos

In [274]:
%%sql
ALTER TABLE blocos_partidos_camara
    ALTER COLUMN id_bloco_partido TYPE BIGINT;

ALTER TABLE blocos_partidos_camara
    ADD CONSTRAINT pk_bloco_partido
    PRIMARY KEY (id_bloco_partido);

Running query in 'duck'

Success


### 3.1.16 Deputados e frentes

In [275]:
%%sql
ALTER TABLE deputados_frentes_camara
    ALTER COLUMN id_deputado_frente TYPE BIGINT;

ALTER TABLE deputados_frentes_camara
    ADD CONSTRAINT pk_deputado_frente
    PRIMARY KEY (id_deputado_frente);

Running query in 'duck'

Success


### 3.1.17 Histórico de deputados

In [276]:
%%sql
ALTER TABLE deputados_historico_camara
    ALTER COLUMN id_deputado_historico TYPE BIGINT;

ALTER TABLE deputados_historico_camara
    ADD CONSTRAINT pk_deputado_historico
    PRIMARY KEY (id_deputado_historico);

Running query in 'duck'

Success


### 3.1.18 Deputados e órgãos

In [277]:
%%sql
ALTER TABLE deputados_orgaos_camara
    ALTER COLUMN id_deputado_orgao TYPE BIGINT;

ALTER TABLE deputados_orgaos_camara
    ADD CONSTRAINT pk_deputado_orgao
    PRIMARY KEY (id_deputado_orgao);

Running query in 'duck'

Success


### 3.1.19 Eventos e órgãos

In [278]:
%%sql
ALTER TABLE eventos_orgaos_camara
    ALTER COLUMN id_evento_orgao TYPE BIGINT;

ALTER TABLE eventos_orgaos_camara
    ADD CONSTRAINT pk_evento_orgao
    PRIMARY KEY (id_evento_orgao);

Running query in 'duck'

Success


### 3.1.20 Partidos e líderes

In [279]:
%%sql
ALTER TABLE partidos_lideres_camara
    ALTER COLUMN id_partido_lider TYPE BIGINT;

ALTER TABLE partidos_lideres_camara
    ADD CONSTRAINT pk_partido_lider
    PRIMARY KEY (id_partido_lider);

Running query in 'duck'

Success


### 3.1.21 Partidos e membros

In [280]:
%%sql
ALTER TABLE partidos_membros_camara
    ALTER COLUMN id_partido_membro TYPE BIGINT;

ALTER TABLE partidos_membros_camara
    ADD CONSTRAINT pk_partido_membro
    PRIMARY KEY (id_partido_membro);

Running query in 'duck'

Success


## 3.2 Deduplicação de tabelas relacionais

### 3.2.1 Orientações

In [281]:
%%sql
SELECT a.*
FROM orientacoes_camara a
JOIN (
    SELECT sigla_partido_bloco, id_votacao
    FROM orientacoes_camara
    GROUP BY sigla_partido_bloco, id_votacao
    HAVING COUNT(*) > 1
) dups
ON  a.sigla_partido_bloco = dups.sigla_partido_bloco
AND a.id_votacao = dups.id_votacao
ORDER BY a.id_votacao, a.sigla_partido_bloco;


Running query in 'duck'

id_orientacao,id_votacao,sigla_partido_bloco,orientacao_voto,cod_partido_bloco,cod_tipo_lideranca,uri_partido_bloco,year_snapshot
40598,2322436-12,AVANTE,Sim,36898,P,https://dadosabertos.camara.leg.br/api/v2/partidos/36898,2021
47965,2322436-12,AVANTE,Sim,36898,P,https://dadosabertos.camara.leg.br/api/v2/partidos/36898,2022
48244,2322436-12,CIDADANIA,Sim,37905,P,https://dadosabertos.camara.leg.br/api/v2/partidos/37905,2022
40914,2322436-12,CIDADANIA,Sim,37905,P,https://dadosabertos.camara.leg.br/api/v2/partidos/37905,2021
49055,2322436-12,Governo,,None,B,None,2022
41857,2322436-12,Governo,,None,B,None,2021
41229,2322436-12,MDB,Sim,36899,P,https://dadosabertos.camara.leg.br/api/v2/partidos/36899,2021
48518,2322436-12,MDB,Sim,36899,P,https://dadosabertos.camara.leg.br/api/v2/partidos/36899,2022
38682,2322436-12,Maioria,Sim,None,B,None,2021
46116,2322436-12,Maioria,Sim,None,B,None,2022


In [282]:
%%sql
SELECT SUM(c) AS total_duplicadas
FROM (
    SELECT COUNT(*) - 1 AS c
    FROM orientacoes_camara
    GROUP BY id_votacao, sigla_partido_bloco
    HAVING COUNT(*) > 1
) t;

Running query in 'duck'

total_duplicadas
27


In [283]:
%%sql
SELECT COUNT(*) FROM orientacoes_camara;

Running query in 'duck'

count_star()
59769


In [284]:
%%sql
DELETE FROM orientacoes_camara
WHERE rowid IN (
    SELECT rowid
    FROM (
        SELECT 
            rowid,
            ROW_NUMBER() OVER (
                PARTITION BY id_votacao, sigla_partido_bloco 
                ORDER BY rowid
            ) AS rn
        FROM orientacoes_camara
    ) t
    WHERE rn > 1
);


Running query in 'duck'

Count
27


In [285]:
%%sql
SELECT COUNT(*) FROM orientacoes_camara;

Running query in 'duck'

count_star()
59742


### 3.2.2 Tramitações

In [286]:
%%sql
SELECT a.*
FROM tramitacoes_camara a
JOIN (
    SELECT id_proposicao, despacho, sequencia
    FROM tramitacoes_camara
    GROUP BY id_proposicao, despacho, sequencia
    HAVING COUNT(*) > 1
) dups
ON  a.id_proposicao = dups.id_proposicao
AND a.sequencia = dups.sequencia
AND a.despacho = dups.despacho
ORDER BY a.id_proposicao, a.sequencia, a.despacho;


Running query in 'duck'

id_tramitacao,id_proposicao,ambito,apreciacao,cod_situacao,cod_tipo_tramitacao,data_hora,descricao_situacao,descricao_tramitacao,despacho,regime,sequencia,sigla_orgao,uri_orgao,uri_ultimo_relator,year_snapshot
769040,15508,Regimental,Proposição Sujeita à Apreciação Conclusiva pelas Comissões - Art. 24 II,923,504,2012-04-02 11:11:00,Arquivada,Notificação de Apensação,Apense-se a este(a) o(a) PL-3440/2012.,"Ordinário (Art. 151, III, RICD)",112,MESA,https://dadosabertos.camara.leg.br/api/v2/orgaos/4,https://dadosabertos.camara.leg.br/api/v2/deputados/74039,2024
769097,15508,Regimental,Proposição Sujeita à Apreciação Conclusiva pelas Comissões - Art. 24 II,923,504,2012-04-02 11:08:00,Arquivada,Notificação de Apensação,Apense-se a este(a) o(a) PL-3440/2012.,"Ordinário (Art. 151, III, RICD)",112,MESA,https://dadosabertos.camara.leg.br/api/v2/orgaos/4,https://dadosabertos.camara.leg.br/api/v2/deputados/74039,2024
584271,47622,Regimental,Proposição Sujeita à Apreciação do Plenário,None,5,2015-03-30 18:46:00,None,Não Informado,Matéria não apreciada em face do encerramento da Sessão.,"Urgência (Art. 155, RICD)",16,PLEN,https://dadosabertos.camara.leg.br/api/v2/orgaos/180,None,2023
586099,47622,Regimental,Proposição Sujeita à Apreciação do Plenário,None,5,2015-03-26 09:00:00,None,Não Informado,Matéria não apreciada em face do encerramento da Sessão.,"Urgência (Art. 155, RICD)",16,PLEN,https://dadosabertos.camara.leg.br/api/v2/orgaos/180,None,2023
67160,332450,Regimental,Proposição Sujeita à Apreciação do Plenário,902,100,2006-08-09 17:25:00,Aguardando Criação de Comissão Temporária,Apresentação de Proposição,"Apresentação do PL 7419/2006, do Senado Federal - Luiz Pontes, que ""altera a Lei nº 9.656, de 3 de junho de 1998, que dispõe sobre os planos e seguros privados de assistência à saúde.""","Urgência (Art. 155, RICD)",1,PLEN,https://dadosabertos.camara.leg.br/api/v2/orgaos/180,None,2020
67157,332450,Regimental,Proposição Sujeita à Apreciação do Plenário,924,100,2006-08-09 17:25:00,Pronta para Pauta,Apresentação de Proposição,"Apresentação do PL 7419/2006, do Senado Federal - Luiz Pontes, que ""altera a Lei nº 9.656, de 3 de junho de 1998, que dispõe sobre os planos e seguros privados de assistência à saúde.""","Urgência (Art. 155, RICD)",1,PLEN,https://dadosabertos.camara.leg.br/api/v2/orgaos/180,None,2020
67067,332450,Regimental,Proposição Sujeita à Apreciação do Plenário,902,194,2011-10-04 00:00:00,Aguardando Criação de Comissão Temporária,Apresentação de Requerimento,"Apresentação do Requerimento de Apensação n. 3395/2011, pelo Deputado Bruno Araújo (PSDB-PE), que: ""Requer a apensação do Projeto de Lei 7419, de 2006, ao Projeto de Lei 4076, de 2001"".","Urgência (Art. 155, RICD)",40,PLEN,https://dadosabertos.camara.leg.br/api/v2/orgaos/180,https://dadosabertos.camara.leg.br/api/v2/deputados/74173,2020
67062,332450,Regimental,Proposição Sujeita à Apreciação do Plenário,924,194,2011-10-04 00:00:00,Pronta para Pauta,Apresentação de Requerimento,"Apresentação do Requerimento de Apensação n. 3395/2011, pelo Deputado Bruno Araújo (PSDB-PE), que: ""Requer a apensação do Projeto de Lei 7419, de 2006, ao Projeto de Lei 4076, de 2001"".","Urgência (Art. 155, RICD)",40,PLEN,https://dadosabertos.camara.leg.br/api/v2/orgaos/180,https://dadosabertos.camara.leg.br/api/v2/deputados/74173,2020
67057,332450,Regimental,Proposição Sujeita à Apreciação do Plenário,902,194,2011-10-04 00:00:00,Aguardando Criação de Comissão Temporária,Apresentação de Requerimento,"Apresentação do Requerimento de Apensação n. 3397/2011, pelo Deputado Bruno Araújo (PSDB-PE), que: ""Requer a apensação do Projeto de Lei 7419, de 2006, ao Projeto de Lei 4076, de 2001."".","Urgência (Art. 155, RICD)",41,PLEN,https://dadosabertos.camara.leg.br/api/v2/orgaos/180,https://dadosabertos.camara.leg.br/api/v2/deputados/74173,2020
67052,332450,Regimental,Proposição Sujeita à Apreciação do Plenário,924,194,2011-10-04 00:00:00,Pronta para Pauta,Apresentação de Requerimento,"Apresentação

In [287]:
%%sql
SELECT SUM(c) AS total_duplicadas
FROM (
    SELECT COUNT(*) - 1 AS c
    FROM tramitacoes_camara
    GROUP BY id_proposicao, despacho, sequencia
    HAVING COUNT(*) > 1
) t;

Running query in 'duck'

total_duplicadas
762


In [288]:
%%sql
DELETE FROM tramitacoes_camara
WHERE rowid IN (
    SELECT rowid
    FROM (
        SELECT 
            rowid,
            ROW_NUMBER() OVER (
                PARTITION BY id_proposicao, despacho, sequencia 
                ORDER BY rowid
            ) AS rn
        FROM tramitacoes_camara
    ) t
    WHERE rn > 1
);


Running query in 'duck'

Count
762


### 3.2.3 Votos

In [289]:
%%sql
SELECT a.*
FROM votos_camara a
JOIN (
    SELECT id_votacao, id_deputado, data_hora
    FROM votos_camara
    GROUP BY id_votacao, id_deputado, data_hora
    HAVING COUNT(*) > 1
) dups
ON  a.id_deputado = dups.id_deputado
AND a.id_votacao = dups.id_votacao
AND a.data_hora = dups.data_hora
ORDER BY a.id_deputado, a.id_votacao;

Running query in 'duck'

id_voto,id_votacao,id_deputado,tipo_voto,data_hora,year_snapshot
719568,2322436-12,68720,Sim,2022-05-24 20:46:51,2021
719569,2322436-12,68720,Sim,2022-05-24 20:46:51,2022
719570,2322436-12,69871,Sim,2022-05-24 20:45:44,2021
719571,2322436-12,69871,Sim,2022-05-24 20:45:44,2022
719572,2322436-12,72442,Sim,2022-05-24 20:46:25,2021
719573,2322436-12,72442,Sim,2022-05-24 20:46:25,2022
719574,2322436-12,73433,Sim,2022-05-24 20:46:34,2021
719575,2322436-12,73433,Sim,2022-05-24 20:46:34,2022
719576,2322436-12,73441,Sim,2022-05-24 20:46:57,2021
719577,2322436-12,73441,Sim,2022-05-24 20:46:57,2022


In [290]:
%%sql
SELECT SUM(c) AS total_duplicadas
FROM (
    SELECT COUNT(*) - 1 AS c
    FROM votos_camara
    GROUP BY id_votacao, id_deputado, data_hora
    HAVING COUNT(*) > 1
) t;

Running query in 'duck'

total_duplicadas
380


In [291]:
%%sql
DELETE FROM votos_camara
WHERE rowid IN (
    SELECT rowid
    FROM (
        SELECT 
            rowid,
            ROW_NUMBER() OVER (
                PARTITION BY id_votacao, id_deputado, data_hora
                ORDER BY rowid
            ) AS rn
        FROM votos_camara
    ) t
    WHERE rn > 1
);

Running query in 'duck'

Count
380


### 3.2.4 Deputados e frentes

In [292]:
%%sql
SELECT a.*
FROM deputados_frentes_camara a
JOIN (
    SELECT id_deputado, id_frente
    FROM deputados_frentes_camara
    GROUP BY id_deputado, id_frente
    HAVING COUNT(*) > 1
) dups
ON  a.id_deputado = dups.id_deputado
AND a.id_frente = dups.id_frente
ORDER BY a.id_deputado, a.id_frente;

Running query in 'duck'

id_deputado_frente,id_deputado,id_frente,year_snapshot
1449,74141,350,2020
1939,74141,350,2021
522,74141,351,2021
28,74141,351,2020
747,74141,353,2020
1238,74141,353,2021
3524,74141,354,2020
4009,74141,354,2021
2837,74141,355,2020
3324,74141,355,2021


In [293]:
%%sql
SELECT SUM(c) AS total_duplicadas
FROM (
    SELECT COUNT(*) - 1 AS c
    FROM deputados_frentes_camara
    GROUP BY id_deputado, id_frente
    HAVING COUNT(*) > 1
) t;

Running query in 'duck'

total_duplicadas
3299


In [294]:
%%sql
DELETE FROM deputados_frentes_camara
WHERE rowid IN (
    SELECT rowid
    FROM (
        SELECT 
            rowid,
            ROW_NUMBER() OVER (
                PARTITION BY id_deputado, id_frente
                ORDER BY rowid
            ) AS rn
        FROM deputados_frentes_camara
    ) t
    WHERE rn > 1
);

Running query in 'duck'

Count
3299


### 3.2.5 Pautas de eventos

In [295]:
%%sql
SELECT a.*
FROM eventos_pauta_camara a
JOIN (
    SELECT id_evento, ordem
    FROM eventos_pauta_camara
    GROUP BY id_evento, ordem
    HAVING COUNT(*) > 1
) dups
ON  a.id_evento = dups.id_evento
AND a.ordem = dups.ordem
ORDER BY a.id_evento, a.ordem;

Running query in 'duck'

id_pauta,id_evento,cod_regime,ordem,id_proposicao,id_relator,year_snapshot
80,59265,21.0,14,2236343,164360,2020
81,59265,21.0,14,2236343,164360,2020
82,59265,21.0,14,2236343,164360,2020
90,59275,15.0,1,2236326,204556,2020
91,59275,15.0,1,2236326,204556,2020
115,59290,100.0,12,2222884,None,2020
116,59290,100.0,12,2222884,None,2020
117,59290,100.0,12,2222884,None,2020
118,59290,100.0,12,2222884,None,2020
121,59298,100.0,1,2222884,None,2020


In [296]:
%%sql
SELECT SUM(c) AS total_duplicadas
FROM (
    SELECT COUNT(*) - 1 AS c
    FROM eventos_pauta_camara
    GROUP BY id_evento, ordem, id_proposicao
    HAVING COUNT(*) > 1
) t;

Running query in 'duck'

total_duplicadas
4060


In [297]:
%%sql
DELETE FROM eventos_pauta_camara
WHERE rowid IN (
    SELECT rowid
    FROM (
        SELECT 
            rowid,
            ROW_NUMBER() OVER (
                PARTITION BY id_evento, ordem, id_proposicao
                ORDER BY rowid
            ) AS rn
        FROM eventos_pauta_camara
    ) t
    WHERE rn > 1
);

Running query in 'duck'

Count
4060


# 4. Limpeza

In [298]:
conn.close()